In [1]:
import h5py
%matplotlib inline
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patches as patches
import sys
import shutil
import glob
import optparse
import os
import json
import pandas as pd
import numpy as np
import pylab as pl
import scipy.stats as stats
import seaborn as sns
sys.path.append('/n/coxfs01/cechavarria/repos/2p-pipeline/')

In [ ]:
class struct: pass

opts = struct()
opts.rootdir = '/n/coxfs01/2p-data'
opts.animalid = 'JC110'
opts.session = '20190909'
opts.acquisition = 'FOV1_zoom4p0x'
opts.traceid = 'traces102_s2p'
opts.combined_run = 'scenes_combined'
opts.motion_thresh = '5'
opts.filter_crit = 'simple_tstat'
opts.filter_thresh = 4

In [ ]:
filter_crit = opts.filter_crit
filter_thresh = int(opts.filter_thresh)

traceid = '%s_s2p'%(opts.traceid)
#% Set up paths:    
acquisition_dir = os.path.join(opts.rootdir, opts.animalid, opts.session, opts.acquisition)
traceid_dir = os.path.join(acquisition_dir, opts.combined_run,'traces',traceid)

run_dir = traceid_dir.split('/traces')[0]
trace_arrays_dir = os.path.join(traceid_dir,'files')
paradigm_dir = os.path.join(acquisition_dir, opts.combined_run, 'paradigm')

#output directory for figure
fig_folder = 'motion_%s_%s_%d'%(opts.motion_thresh,filter_crit,filter_thresh)
fig_out_dir = os.path.join(traceid_dir,'figures','filtered_traces',fig_folder)
if not os.path.exists(fig_out_dir):
    os.makedirs(fig_out_dir)

#read trace file file
parsedtraces_filepath = glob.glob(os.path.join(traceid_dir, 'files','parsedtraces*'))[0]
file_grp = h5py.File(parsedtraces_filepath, 'r')

curr_slice = 'Slice01'#hard-code planar data for now
pix_cell_array = np.array(file_grp[curr_slice]['traces']['np_subtracted'])
nrois = pix_cell_array.shape[2]

if 's2p_cell_rois' in file_grp.attrs.keys():
    cell_rois = file_grp.attrs['s2p_cell_rois']
else:
    cell_rois = np.arange(nrois)




pre_frames = file_grp.attrs['pre_frames']
post_frames = file_grp.attrs['post_frames']
stim_frames = file_grp.attrs['stim_frames']
#to get baseline index with [0:pre_frames]
#to get stim period do [pre_frames:pre_frames+stim_frames+1]


frames_tsec = np.array(file_grp[curr_slice]['frames_tsec'])
#get raw pixel value arrays



file_grp.close()

#get file with trial-by-trial responses
responses_dir = os.path.join(acquisition_dir, opts.combined_run,'responses',traceid)
data_array_dir = os.path.join(responses_dir, 'data_arrays')

resp_array_fn = 'trial_response_array_motion_%s.hdf5'%(opts.motion_thresh)
resp_array_filepath = os.path.join(data_array_dir, resp_array_fn)
resp_grp = h5py.File(resp_array_filepath, 'r')


curr_slice = 'Slice01'#hard,coding for now

#unpack
if filter_crit == 'zscore':
    filter_crit_matrix_trials = np.array(resp_grp['/'.join([curr_slice, 'responses' ,filter_crit])])
    filter_crit_matrix_trials = filter_crit_matrix_trials[:,:,cell_rois]
    filter_crit_matrix = np.squeeze(np.mean(filter_crit_matrix_trials,0))
elif filter_crit == 'simple_tstat':
    filter_crit_matrix = np.array(resp_grp['/'.join([curr_slice, 'responses' ,filter_crit])])


#consider only cell rois and trials that pass threshold
pix_cell_array = pix_cell_array[:,:,cell_rois]
ntrials,ntpts,nrois = pix_cell_array.shape

resp_grp.close()




#consider a config active if at least one of theversion of an image evoked a response above threshol
thresh = filter_thresh
thresh_matrix = filter_crit_matrix>thresh
filter_matrix = np.zeros((thresh_matrix.shape))
for ridx in range(nrois):
    for idx in range(0,thresh_matrix.shape[0],3):
            if np.sum(thresh_matrix[idx:idx+3,ridx])>0:
                filter_matrix[idx:idx+3,ridx] = 1

In [ ]:
#load trial info
stimconfig_fn = 'trial_conditions.hdf5'
stimconfig_filepath = os.path.join(paradigm_dir, 'files', stimconfig_fn)
config_grp = h5py.File(stimconfig_filepath, 'r')

trial_config = np.array(config_grp['trial_config'])
trial_cond = np.array(config_grp['trial_cond'])
trial_img = np.array(config_grp['trial_img'])

config_grp.close()

#for each roi, first average across trials per config, then normalize across configs
#-should result in higher values, also noticed bigger differences in values

nconfigs = len(np.unique(trial_config))
print('ROIs:%i'%(nrois))
print('Trials:%i'%(ntrials))


norm_roi_df_array = np.empty((ntpts,nconfigs,nrois))
norm_array_filt = np.ones((norm_roi_df_array.shape))*np.nan
config_cond = np.empty((nconfigs,))
config_img = np.empty((nconfigs,))

for ridx in range(nrois):
#ridx = 0
    roi_df_array = np.empty((ntrials,ntpts))
    for tidx in range(ntrials):

        #get trial timecourse
        pix_cell = np.squeeze(pix_cell_array[tidx,:,ridx].squeeze())

        #get baseline
        base_cell = pix_cell[0:pre_frames]

        #calculate df
        df_cell = pix_cell - np.mean(base_cell)

        #store in array
        roi_df_array[tidx,:] = df_cell


    mean_df_array = np.empty((nconfigs,ntpts))
    for cfg_idx in np.unique(trial_config):


        cfg_key = 'config%03d'%(cfg_idx)
        tidx = np.where(trial_config == cfg_idx)[0]
        config_cond[int(cfg_idx)] = trial_cond[tidx][0]
        config_img[int(cfg_idx)] = trial_img[tidx][0]

        mean_df_array[int(cfg_idx),:] = np.nanmean(roi_df_array[tidx,:],0)

    #normalize traces across configs
    norm_roi_df_array[:,:,ridx] = np.transpose(mean_df_array/np.nanmax(mean_df_array.flatten()))

    #filter out traces to unresponsive conditions within neuron
    for cfg_idx in np.unique(trial_config):
        cfg_idx = int(cfg_idx)
        if filter_matrix[cfg_idx,ridx]:
            norm_array_filt[:,cfg_idx,ridx] = norm_roi_df_array[:,cfg_idx,ridx]

#figure out some activity details
num_active_rois = np.sum(np.sum(filter_matrix,0)>0)
frac_active_rois = num_active_rois/float(len(cell_rois))
print('# active rois = %i'%(num_active_rois))
print('frac active rois = %.04f'%(frac_active_rois))